### Libraries

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from tqdm import tqdm
import re

### Scraping

In [3]:
headers = {"User-Agent": "Mozilla/5.0"}
cookies = {"CONSENT": "YES+cb.20211025-07-p0.en+FX+410"}

In [21]:
#### Going through the first gen of SCPs ####
scp_number_list=[]
scp_text_list=[]
scp_rating_list=[]
scp_discussion_list=[]

for i in tqdm(range(2,2701)): # We limit the scrap to 2700 SCPs because of recent website changes affecting recent SCPs
    # condition to put the accurate scp number in the link, we use the scp_number_prev and next to clean the footnote
    if i>=2 and i<10:
        scp_number = "00"+str(i)
        scp_number_prev = "00"+str(i-1)
        scp_number_next = "00"+str(i+1)
        url = ("https://scp-wiki.wikidot.com/scp-"+scp_number)
    elif i>=10 and i<100:
        scp_number = "0"+str(i)
        scp_number_prev = "0"+str(i-1)
        scp_number_next = "0"+str(i+1)
        url = ("https://scp-wiki.wikidot.com/scp-"+scp_number)
    else:
        scp_number = str(i)
        scp_number_prev = str(i-1)
        scp_number_next = str(i+1)
        url = ("https://scp-wiki.wikidot.com/scp-"+scp_number)
    # initializing our soup config
    page = requests.get(url, cookies=cookies, headers=headers)
    soup = BeautifulSoup(page.content,'html.parser')    
   
    ##### Getting the text and the SCP number #####
    div = soup.find("div", {"id": "page-content"})
    tag_types = ["p", "ul", "li"]
    my_text=[]
    # condition making sure we don't pick the stuff 
    if soup.find("div", {"class": "licensebox"}): #returns none, some pages have a license part, some not
        for item in range(len(div.find_all(tag_types))-6): # if license box, we  remove everything
            my_text.append(div.find_all(tag_types)[item].get_text().strip())
    else:
        for item in range(len(div.find_all(tag_types))-2): # if not we remove just the necessary
            my_text.append(div.find_all(tag_types)[item].get_text().strip())
    # making sure we the "item #: SCP-xxx" is the start of the text if possible        
    try:
        start_index = my_text.index("Item #: SCP-"+scp_number)
        my_text = my_text[start_index:]
    except:
        pass
    # making sure we don't pick the footnote navigation bar at the end
    footnote = "« SCP-"+scp_number_prev+" | SCP-"+scp_number+" | SCP-"+scp_number_next+" »"
    try:
        footnote_index = my_text.index(footnote)
        my_text = my_text[:footnote_index]
    except:
        pass
    #####
    
    ##### Getting the rating and the discussion #####
    div = soup.find("div", {"id": "page-options-bottom"})
    # Getting the rating
    rating = div.find('a', id='pagerate-button').text
    rating = rating.split('(', 1)[1].split(')')[0]
    # Getting the discussion
    discussion = (div.find('a', id='discuss-button')).text
    discussion = discussion.split('(', 1)[1].split(')')[0]
    #####
    
    # storing the results
    scp_number_list.append(scp_number)
    scp_text_list.append(my_text)
    scp_rating_list.append(rating)
    scp_discussion_list.append(discussion)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.89it/s]


### Storing

In [22]:
scp_df = pd.DataFrame(columns = ['SCP Number', 'SCP Text', 'Ratings', 'Discussions'])
scp_df.iloc[:,0] = scp_number_list
scp_df.iloc[:,1] = scp_text_list
scp_df.iloc[:,2] = scp_rating_list
scp_df.iloc[:,3] = scp_discussion_list

In [23]:
scp_df["Ratings"] = scp_df["Ratings"].astype('int64')
scp_df["Discussions"] = scp_df["Discussions"].astype('int64')

### Exporting

In [204]:
scp_df.to_csv('scp_20112021_export_4.csv')

### Notes

In [197]:
# Final table structure:

# 1/ series
# 2/ scp number
# 3/ text
# (optional) 3.5/ media if any 
# 4/ ratings
# 5/ nbr discussions
# 6/ tags
# 7/ nbr page revisions
# 8/ date last edit

In [198]:
# Final Questions to answer:

## Sentiment through the SCPs
## Most active years
## sizes (just get the cm)
## the people/orga in the SCP
## the people/orga in the SCP and how they evolve through the SCP numbers
## How these people/orga relate to each other -> Trying to make a lore out of everything
## Is there a pattern to redactions ?
## Is there a correlation with the ratings and: the theme of the SCP, the ratings, tags, sentiments and the object class ?
## Keter vs Euclid vs Safe: What are their main textual patterns ?
## Generate novel text

In [ ]:
## What to improve:
    # Better scraping for earlier and for advanced series 
    # level 1,2,3,4 is not detected well: Making 'groups'
    # we remove some words that need to stick together ... Or we seperate in token words that need to stick together otherwise they don't make sense 
    # Make functions instead
    # Using SciBERT 